## Deidentification of all Datasets

#### - This script cleans the input datasets and deidentifies them with a randomly generated ID.
#### - Note, each time you run this it will generate a new randomized ID list to de-identify the data.
#### - Dates will not be adjsuted to "DAYS" - This happens downstream because dates are used as links

In [1]:
# Import Modules

import numpy as np
import time as time
import pandas as pd
import datetime as dt
from patsy import dmatrices
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
%matplotlib inline


In [2]:
# Load original Microbiology data


df1 = pd.read_csv('/Users/chackk02/Documents/Epic_Reports/Micro Bacterium Report_Harm (Jan 2010 to Dec 2010) 2017-09-19-16-09-00.txt.ttx',
                  sep = "\t",
                  quotechar='"',
                  skiprows=[0],
                  header = None,
                  low_memory=False)

df2 = pd.read_csv('/Users/chackk02/Documents/Epic_Reports/Micro Bacterium Report_Harm (Jan 2011 to Mar 2012) 2017-09-19-16-09-10.txt.ttx',
                  sep = "\t",
                  quotechar='"',
                  skiprows=[0],
                  header = None,
                  low_memory=False)

df3 = pd.read_csv('/Users/chackk02/Documents/Epic_Reports/Micro Bacterium Report_Harm (Apr 2012 to Mar 2013) 2017-04-04-10-17-15.txt.ttx',
                 sep = "\t",
                 quotechar='"',
                 skiprows=[0],
                 header = None,
                 low_memory=False)

df4 = pd.read_csv('/Users/chackk02/Documents/Epic_Reports/Micro Bacterium Report_Harm (Apr 2013 to Mar 2014) 2017-04-04-13-56-59.txt.ttx',
                 sep = "\t",
                 quotechar='"',
                 skiprows=[0],
                 header = None,
                 low_memory=False)

df5 = pd.read_csv('/Users/chackk02/Documents/Epic_Reports/Micro Bacterium Report_Harm (Apr 2014 to Mar 2015) 2017-04-04-15-03-02.txt.ttx',
                 sep = "\t",
                 quotechar='"',
                 skiprows=[0],
                 header = None,
                 low_memory=False)

df6 = pd.read_csv('/Users/chackk02/Documents/Epic_Reports/Micro Bacterium Report_Harm (Apr 2015 to Mar 2016) 2017-04-04-19-36-01.txt.ttx',
                 sep = "\t",
                 quotechar='"',
                 skiprows=[0],
                 header = None,
                 low_memory=False)

df7 = pd.read_csv('/Users/chackk02/Documents/Epic_Reports/Micro Bacterium Report_Harm (Apr 2016 to YTD) 2017-04-04-19-38-06.txt.ttx',
                 sep = "\t",
                 quotechar='"',
                 skiprows=[0],
                 header = None,
                 low_memory=False)

# Combine the separate files together
dataframes = [df1, df2, df3, df4, df5, df6, df7]
microlab = pd.concat(dataframes)

# Relabel the columns
microlab.columns = [
    "MRN",
    "Unit_at_Order",
    "Unit",
    "Room",
    "Bed",
    "ORDERING_DATE",
    "SPECIMEN_TAKEN_DATE",
    "ACC_NUM",
    "Source1",
    "Source2",
    "Line",
    "Organism",
    "Abx Name",
    "ADMIT_DATE",
    "Suscept",
    "SENSITIVITY_VALUE",
    "Order ID",
    "Proc Code",
    "Procedure",
    "Result"
    ]

# Fix the date format
microlab['ORDERING_DATE'] = pd.to_datetime(microlab['ORDERING_DATE'], format = "%m/%d/%y")
microlab['SPECIMEN_TAKEN_DATE'] = microlab['SPECIMEN_TAKEN_DATE'].str.split(' ').str[0]
microlab['SPECIMEN_TAKEN_DATE'] = pd.to_datetime(microlab['SPECIMEN_TAKEN_DATE'], format = "%m/%d/%Y")
microlab['ADMIT_DATE'] = microlab['ADMIT_DATE'].str.split(' ').str[0]
microlab['ADMIT_DATE'] = pd.to_datetime(microlab['ADMIT_DATE'], format = "%m/%d/%Y")

# Drop rows with missing information
microlab = microlab.dropna(subset=['MRN', 'SPECIMEN_TAKEN_DATE', 'ACC_NUM', 'Organism', 'Abx Name'])


/Applications/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (1,11,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Applications/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Applications/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
# Load Encounters data

df1 = pd.read_csv('/Users/chackk02/Documents/Epic_Reports/Patients - Encounters (Kieran I Chacko) 2017-10-03-14-23-25.csv',
                  sep = ",",
                  quotechar='"',
                  skiprows=[0],
                  header = None,
                  low_memory=False)

encounters = df1

# Relabel the columns
encounters.columns = [
    "PAT_ENC_CSN_ID",
    "ENCOUNTER_DATE",
    "VISIT_ID",
    "DEPARTMENT_NAME",
    "ENCOUNTER_TYPE",
    "MRN",
    "AGE",
    "SEX",
    "BMI",
    "SOCIAL_HX_TOBACCO_USER",
    "HOSP_ADMSN_TIME",
    "HOSP_DISCHRG_TIME",
    "PREVIOUS_ADMISSION_DATE",
    "PREVIOUS_DISCHARGE_DATE",
    "TPN_IN_CURRENT_VISIT",
    "COLORECTAL_CANCER_IN_CURRENT_VISIT",
    "UC_IN_CURRENT_VISIT",
    "CROHNS_IN_CURRENT_VISIT",
    "IBD_IN_CURRENT_VISIT",
    "IBS_IN_CURRENT_VISIT"
    ]

# Fix the date format
encounters.ix[encounters['HOSP_ADMSN_TIME'].isnull(), 'HOSP_ADMSN_TIME'] = encounters['ENCOUNTER_DATE']   # There are many non-admits that have NAN in this field. Let's assume that they are same day visits.
encounters.ix[encounters['HOSP_DISCHRG_TIME'].isnull(), 'HOSP_DISCHRG_TIME'] = encounters['ENCOUNTER_DATE']   # There are many non-admits that have NAN in this field. Let's assume that they are same day visits.
encounters['ENCOUNTER_DATE'] = pd.to_datetime(encounters['ENCOUNTER_DATE'], format = "%m/%d/%Y")
encounters['HOSP_ADMSN_TIME'] = encounters['HOSP_ADMSN_TIME'].str.split(' ').str[0]
encounters['HOSP_ADMSN_TIME'] = pd.to_datetime(encounters['HOSP_ADMSN_TIME'], format = "%m/%d/%Y")
encounters['HOSP_DISCHRG_TIME'] = encounters['HOSP_DISCHRG_TIME'].str.split(' ').str[0]
encounters['HOSP_DISCHRG_TIME'] = pd.to_datetime(encounters['HOSP_DISCHRG_TIME'], format = "%m/%d/%Y")

encounters = encounters.sort_values(by = ['HOSP_ADMSN_TIME'])

In [28]:
# Load Medications data

df1 = pd.read_csv('/Users/chackk02/Documents/Epic_Reports/Patients - Meds (Kieran I Chacko) 2017-10-03-14-12-35.csv',
                  sep = ",",
                  quotechar='"',
                  skiprows=[0],
                  header = None,
                  low_memory = False)

df2 = pd.read_csv('../Data/ABX_Keys.txt',
                  sep = "\t",
                  quotechar='"',
                  skiprows=[0],
                  header = None,
                  low_memory = False)

df3 = pd.read_csv('../Data/FREQ_Keys.txt',
                  sep = "\t",
                  quotechar='"',
                  skiprows=[0],
                  header = None,
                  low_memory = False)

medications = df1
ABX = df2
FREQ = df3

medications.columns = ["PAT_ENC_CSN_ID",
               "ENCOUNTER_DATE",
               "VISIT_ID",
               "DEPARTMENT_NAME",
               "ENCOUNTER_TYPE",
               "MRN",
               "ORDER_MED_ID",
               "MEDICATION_ID",
               "MED_NAME",
               "ORDER_START_TIME",
               "ORDER_END_TIME",
               "DOSE",
               "DOSE_UNIT",
               "ROUTE",
               "FREQ",
               "TAKEN_TIME"
                       ]

ABX.columns = ['Antibiotic-Treatment',
              'ANTIBIOTIC_ID',
              'MEDICATION_ID',
              'MED_NAME',
              'ATC_CODE',
               'WHO_DOSE',
               'WHO_DOSE_UNIT',
               'WHO_ROUTE'
              ]

FREQ.columns = ['FREQ',
              'FREQ_ADJ'
               ]

# Merge the Datasets togeter
medications = pd.merge(medications, ABX, on='MEDICATION_ID', how='left')
medications = pd.merge(medications, FREQ, on='FREQ', how='left')

# Fix the date format
medications['ENCOUNTER_DATE'] = pd.to_datetime(medications['ENCOUNTER_DATE'], format = "%m/%d/%Y")
medications['ORDER_START_TIME'] = medications['ORDER_START_TIME'].str.split(' ').str[0]
medications['ORDER_START_TIME'] = pd.to_datetime(medications['ORDER_START_TIME'], format = "%m/%d/%Y", errors = 'coerce')
medications['ORDER_END_TIME'] = medications['ORDER_END_TIME'].str.split(' ').str[0]
medications['ORDER_END_TIME'] = pd.to_datetime(medications['ORDER_END_TIME'], format = "%m/%d/%Y", errors = 'coerce')

# Drop rows with missing information
medications = medications.dropna(subset=['MRN', 'VISIT_ID', 'ENCOUNTER_DATE', 'Antibiotic-Treatment', 'ORDER_START_TIME', 'ORDER_END_TIME'])


In [29]:
# Deidentify the data

# Generate a Unique ID for the MRNs
MRN = microlab['MRN'].unique()
MRN = pd.DataFrame(MRN)
MRN = MRN.sample(frac=1).reset_index(drop=True) # Randomly shuffles all of the rows
MRN.columns = ['MRN']
MRN['ID'] = MRN.index + 1

# Save this file for future use
MRN.to_csv('MRN_Key.txt', sep='\t', index=False, header=False)

# Deidentify Microlab Data
microlab = pd.merge(microlab, MRN, on=['MRN'], how = 'left')
microlab = microlab.drop('MRN', 1)
microlab.to_csv('Micro_Bacterium_Report-Deidentified.txt', sep='\t', index=False)

# Deidentify Encounters Data
encounters = pd.merge(encounters, MRN, on='MRN', how='left')
encounters = encounters.drop('MRN', 1)
encounters.to_csv('Encounters_Report-Deidentified.txt', sep='\t', index=False)

# Deidentify Medications Data
medications = pd.merge(medications, MRN, on='MRN', how='left')
medications = medications.drop('MRN', 1)
medications.to_csv('Medications_Report-Deidentified.txt', sep='\t', index=False)
